In [2]:
import numpy as np 
from scipy import optimize
from scipy.optimize import NonlinearConstraint
from scipy.optimize import minimize
import os

from tqdm import * 
import pickle

import matplotlib.pyplot as plt 
%matplotlib inline 

In [3]:
def sample_stiefel(n,k): 
    
    A = np.random.normal( 0, 1, (n, k) ) 
    
    def normalize(v): 
        return v / np.sqrt(v.dot(v)) 

    n = A.shape[1] 

    A[:, 0] = normalize(A[:, 0])  

    for i in range(1, n): 
        Ai = A[:, i] 
        for j in range(0, i): 
            Aj = A[:, j] 
            t = Ai.dot(Aj) 
            Ai = Ai - t * Aj 
        A[:, i] = normalize(Ai) 
        
    return A 

In [4]:
def func(x): 
    
    return np.sum( np.sin(x) ) + np.exp((x[0]-1) * (x[1]+2))  

# our method, Stiefel's sampling

In [5]:
def stiefel( x, delta, k, n = 500 ): 


    V = sample_stiefel(n,k) 

    gs = [ n/delta/2 * ( func( x + delta * V[:,i]) - func( x - delta * V[:,i]) ) * V[:,i] for i in range(k) ] 

    g = np.mean( np.array( gs ) , axis = 0 ) 
    
    return g

# spherical method

In [6]:
def spherical( x, delta, k, n = 500 ): 
   
    gs = []
    
    for _ in range(k): 
        V = sample_stiefel(n,1)
        gs.append( n/delta/2 * ( func( x + delta * V[:,0]) - func(x - delta * V[:,0]) ) * V[:,0] ) 

    g = np.mean( np.array( gs ) , axis = 0 ) 
    
    return g

# gaussian method

In [7]:
def gaussian( x, delta, k, n = 500 ): 
    
    V = np.random.normal(0,1,(n,k)) 

    gs = [ np.sqrt(n)/delta/2 * ( func( x + delta * V[:,i] / np.sqrt(n) ) \
                                 - func(x - delta * V[:,i] / np.sqrt(n) ) ) * V[:,i] for i in range(k) ] 
    
    g = np.mean( np.array( gs ) , axis = 0 ) 
    
    return g

# rademacher

In [38]:
def rademacher( x, delta, k, n = 500 ): 
   
    gs = []
    
    ind = list(np.random.choice( np.array(range(n)), size = k, replace = False ))
    
    g = np.zeros(n)
    
    for i in range(n): 
        
        if i in ind:
            
            rand = np.random.binomial(1, 0.5)
            
            if rand > 0.5:
                
                v = np.zeros(n)
                v[i] = 1
                
                g[i] = 1/delta * ( func( x + delta* v ) - func(x) )
                
            else:
                
                v = np.zeros(n)
                v[i] = -1
                
                g[i] = -1/delta * ( func( x + delta* v ) - func(x) )
    
    return g 

# comparison-based method

In [39]:
def comp( x, delta, k, sparsity, n = 500 ): 
    
#     zs = []
#     Vs = []
    Us = []
    
    for _ in range(k): 
        
        V = sample_stiefel(n,1).T[0] 
#         Vs.append(V)
    
        C = np.sign( func (x + delta*V) - func (x) ) 
#         zs.append(C) 
        
        Us.append(C * V)
        
#     from scipy import optimize
#     from scipy.optimize import NonlinearConstraint
#     from scipy.optimize import minimize
        
    con2 = lambda x: np.linalg.norm(x) 
    l2 = NonlinearConstraint(con2, -np.inf, 1)
    con1 = lambda x: np.linalg.norm(x, ord = 1) 
    l1 = NonlinearConstraint(con1, -np.inf, np.sqrt(sparsity))
    
    def opt( u ): 
        
        return - np.dot( np.sum( Us, axis = 0 ) , u )
    
    if (sparsity >= np.inf):
    
        g = minimize(opt, np.array( [0]*n ), constraints = l2, tol=1e-6)
        
    else: 
        
        g = minimize(opt, np.array( [0]*n ), constraints = [l2,l1], tol=1e-6)
        
    
    return g 

# comparison-based stiefel

In [40]:
def comp_stiefel( x, delta, k, sparsity, n = 500 ): 
    
#     zs = []
#     Vs = [] 
    Us = []
    
    V = sample_stiefel(n,k)
    
    for m in range(k): 
#         Vs.append(V)
    
        C = np.sign( func (x + delta*V[:,m]) - func (x) ) 
#         zs.append(C) 
        
        Us.append( C * V[:,m]) 
        
#     from scipy import optimize
#     from scipy.optimize import NonlinearConstraint
#     from scipy.optimize import minimize
        
    con2 = lambda x: np.linalg.norm(x) 
    l2 = NonlinearConstraint(con2, -np.inf, 1)
    con1 = lambda x: np.linalg.norm(x, ord = 1) 
    l1 = NonlinearConstraint(con1, -np.inf, np.sqrt(sparsity))
    
    def opt( u ): 
        
        return - np.dot( np.sum( Us, axis = 0 ) , u )
    
    if (sparsity >= np.inf):
    
        g = minimize(opt, np.array( [0]*n ), constraints = l2, tol=1e-6)
        
    else: 
        
        g = minimize(opt, np.array( [0]*n ), constraints = [l2,l1], tol=1e-6)
        
    
    return g 

# entry-wise estimator

In [41]:
def entry( x, delta, n = 500 ): 
    
    g = []
    
    for i in range(n):
        
        ei = np.zeros(n)
        ei[i] = 1
        
        gi = 1./2/delta * ( func (x + delta* ei) - func (x - delta* ei) )
        
        g.append(gi)
    
    return np.array(g) 

# comparison with existing stochastic methods

In [50]:
n = 500

xs = [np.array([0]*n), np.array([np.pi/4.]*n), - np.array([np.pi/2.]*n) ] 

deltas = [ 0.1, 0.01, 0.001 ] 

ks = [ 100,200,300,400,500 ] 

reps = 10 

for i in range(len(xs)):
    
    x = xs[i]
    
    truth = (np.array( np.cos(x) ) + np.array( [(x[1] + 2)* np.exp( (x[0]-1)*(x[1]+2) ) 
                                                 , (x[0] - 1)* np.exp( (x[0]-1)*(x[1]+2) ) ] + [0]*(n-2) ) ) 
    
    for delta in deltas:
        
        for k in ks: 
            
            stiefel_errors = [] 
            spherical_errors = [] 
            gaussian_errors = []
            
            rademacher_errors = []
            
            stiefel_cos_sim = [] 
            
            for _ in range(reps):
                
                stiefel_g = stiefel( x, delta, k ) 
                spherial_g = spherical( x, delta, k ) 
                gaussian_g = gaussian( x, delta, k )  
                
                rademacher_g = rademacher( x, delta, k )  
                
                stiefel_errors.append( np.linalg.norm( stiefel_g - truth )  )
                spherical_errors.append( np.linalg.norm( spherial_g - truth )  )
                gaussian_errors.append( np.linalg.norm( gaussian_g - truth )  )
                
                rademacher_errors.append( np.linalg.norm( rademacher_g - truth ) ) 
                
                stiefel_cos_sim.append( np.dot( stiefel_g, truth ) \
                                       / np.linalg.norm( stiefel_g ) / np.linalg.norm(truth) )
                
                
            pickle.dump( stiefel_errors, 
                        open('./raw_results/stiefel_errors_x{0}_delta{1}_k{2}'.format(i,delta,k), 'wb' ) ) 
            pickle.dump( spherical_errors, 
                        open('./raw_results/spherical_errors_x{0}_delta{1}_k{2}'.format(i,delta,k), 'wb' ) ) 
            pickle.dump( gaussian_errors, 
                        open('./raw_results/gaussian_errors_x{0}_delta{1}_k{2}'.format(i,delta,k), 'wb' ) ) 
            
            pickle.dump( gaussian_errors, 
                        open('./raw_results/rademacher_errors_x{0}_delta{1}_k{2}'.format(i,delta,k), 'wb' ) ) 
            
            pickle.dump( stiefel_cos_sim, 
                        open('./raw_results/stiefel_cos_sim_x{0}_delta{1}_k{2}'.format(i,delta,k), 'wb' ) ) 
                

# comparison with comparison-based estimator

In [25]:
xs = [np.array([0]*500), np.array([np.pi/4.]*500) ] 

deltas = [ 0.1, 0.01, 0.001 ] 

ks = [ 100,200,300,400,500 ] 

sparsity = [ 100, np.inf ] 

reps = 10

for i in range(len(xs)): 
    
    x = xs[i]
    
    truth = (np.array( np.cos(x) ) + np.array( [(x[1] + 2)* np.exp( (x[0]-1)*(x[1]+2) ) 
                                                 , (x[0] - 1)* np.exp( (x[0]-1)*(x[1]+2) ) ] + [0]*(n-2) ) ) 
    
    for delta in deltas:
        
        for k in ks: 
            
            for s in sparsity: 
                
                existing_files = [str(name) for name in os.listdir("./raw_results/")] 
                new_file = 'comp_cos_sim_x{0}_delta{1}_k{2}_sparsity{3}'.format(i, delta, k, s) 
                
#                 print(new_file)
#                 print( new_file not in existing_files ) 
                
                if new_file not in existing_files: 
                
                    comp_cos_sim = [] 

                    for _ in range(reps): 

                        comp_g = comp( x, delta, k, s ) 
                        comp_g = comp_g.x 

                        comp_cos_sim.append( np.dot( comp_g , truth ) \
                                            / np.linalg.norm(comp_g) / np.linalg.norm(truth) )

                    with open('./raw_results/comp_cos_sim_x{0}_delta{1}_k{2}_sparsity{3}'.format(i,delta,k,s), 
                              'wb' ) as f: 

                        pickle.dump( comp_cos_sim, f ) 
        
        

# results of entry-wise estimator

In [86]:
n = 500

xs = [np.array([0]*n), np.array([np.pi/4]*n)] 

deltas = [ 0.1, 0.01, 0.001 ] 

for i in range(len(xs)): 
    
    x = xs[i]
    
    truth = (np.array( np.cos(x) ) + np.array( [(x[1] + 2)* np.exp( (x[0]-1)*(x[1]+2) ) 
                                                 , (x[0] - 1)* np.exp( (x[0]-1)*(x[1]+2) ) ] + [0]*(n-2) ) ) 
    
    for delta in deltas: 
            
        entry_g = entry( x, delta, n =n )  
        
        entry_wise_errors = np.linalg.norm( entry_g - truth )  
        
        print(i,delta,entry_wise_errors) 
                

0 0.1 0.03722295933367299
0 0.01 0.0003724136131194778
0 0.001 3.724154390898286e-06
1 0.1 0.032287116422944684
1 0.01 0.0003225329887275817
1 0.001 3.225438113869768e-06


In [88]:

xs = [np.array([0]*n), np.array([np.pi/4.]*n) ] 

deltas = [ 0.1, 0.01, 0.001 ] 

rep = 10

for i in range(len(xs)): 
    
    x = xs[i]
    
    truth = (np.array( np.cos(x) ) + np.array( [(x[1] + 2)* np.exp( (x[0]-1)*(x[1]+2) ) 
                                                 , (x[0] - 1)* np.exp( (x[0]-1)*(x[1]+2) ) ] + [0]*(n-2) ) ) 
        
    for delta in deltas: 
        
        res = []
            
        for _ in range(rep): 

            stie = stiefel( x, delta, n, n = n ) 

            res.append( np.linalg.norm( stie - truth )  ) 

        print(i,delta,np.mean(res), np.std(res) ) 

0 0.1 0.0002852304664786578 8.27145869150097e-06
0 0.01 2.8750228063198523e-06 8.729980460710897e-08
0 0.001 2.857123846946423e-08 1.0388179997627509e-09
1 0.1 0.0002468521109669915 6.662891121890526e-06
1 0.01 2.5112940783269207e-06 1.7601689049969722e-07
1 0.001 2.5134881612955318e-08 1.5221019074423716e-09
